# Deep Learning for Business Applications course

## TOPIC 2: Introduction to Computer Vision. Image processing with OpenCV. Part 2

### 1. Library installation

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from sklearn.cluster import DBSCAN

### 2. Data preprocess

#### 2.1. Read video file

In [ ]:
DATA_PATH = '/home/jovyan/__DATA/DLBA_F25/topic_02'
WORK_PATH = '.'
IMGS_PATH = 'imgs'
VID_NAME = 'Epic_bullet_trace.mp4'
OUT_FILE = 'test.avi'

In [ ]:
vid_file = f'{DATA_PATH}/{VID_NAME}'
cap = cv2.VideoCapture(vid_file)
frames_cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print('video has {} frames and rate {} fps (frames-per-second)'.format(
    frames_cnt,
    fps
))

#### 2.2. Utility functions

In [ ]:
def str_numbered(i, symbols=5):
    """
    For pretty name of saved video frames.

    :i: number of image
    :symbols: lenght of saved file name
    
    """
    str_num = ''.join([
        '0' * (symbols - len(str(i))),
        str(i)
    ])
    return str_num

In [ ]:
def get_frames(vid_path, start_time, num_frames, save_dir, prefix=''):
    """
    Function takes the path to video
    and saves few frames to the disk.

    :vid_path: path to video file
    :start_time: where to start capturing frames
    :num_frames: ho many frames to save
    :save_dir: path to save to

    """
    files_names = []
    cap = cv2.VideoCapture(vid_path)
    frames_cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_pos = int(start_time * fps)
    end_pos = int(start_pos + num_frames) if num_frames else int(frames_cnt)
    if end_pos <= frames_cnt:
        for frame_num in tqdm(range(start_pos, end_pos)):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            res, frame = cap.read()
            if res:
                file_name = '{}/{}_{}.png'.format(
                    save_dir, 
                    prefix,
                    str_numbered(frame_num)
                )
                cv2.imwrite(file_name, frame)
                files_names.append(file_name)
    else:
        print('out of video lenght')
    c3, c4, c5 = int(cap.get(3)), int(cap.get(4)), cap.get(5)
    cap.release()
    return files_names, frames_cnt, c3, c4, c5

#### 2.3. Save video to frame images

In [ ]:
imgs_dir = f'{WORK_PATH}/{IMGS_PATH}'
os.makedirs(imgs_dir, exist_ok=True)

In [ ]:
files_names, frames_cnt, c3, c4, c5 = get_frames(
    vid_path=vid_file,
    start_time=0, 
    num_frames=None, 
    save_dir=imgs_dir, 
    prefix='frame'
)
len(files_names)

### 3. Finding trace

About the [phenomenon of Vapor trail and Bullet trace](https://snipercountry.com/bullet-trail/).

#### 3.1. Use OpenCV basics

In [ ]:
def get_img(file_path, canny=True):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    assert img is not None, 'file could not be read, check if file exists'
    if canny:
        return cv2.Canny(img, 100, 200, 3)
    else:
        return img


def img_mean(files_names, frame_num, cover_l, cover_r, canny=False):
    files_l = files_names[max(0, frame_num - cover_l):frame_num]
    files_r = files_names[frame_num + 1:min(len(files_names), frame_num + cover_r + 1)]
    files_l.extend(files_r)
    frames = []
    for file_name in files_l:
        frames.append(get_img(file_name, canny=canny))
    mean_frame = np.mean(frames, axis=0).astype(dtype=np.uint8)
    return mean_frame


def img_diff(img1, img2, mask, psize=.1, quantile=.95):
    img = cv2.absdiff(
        cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY), 
        cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    )
    size = int(psize * np.mean(img.shape))
    kernel = np.ones((size, size), np.float32) / (size * size)
    mask = cv2.filter2D(mask, -1, kernel)
    mask = cv2.bitwise_not(mask)
    img = cv2.bitwise_and(img, mask)
    img = cv2.filter2D(img, -1, kernel)
    th, img = cv2.threshold(
        img, 
        np.quantile(img, quantile), 
        np.max(img), 
        cv2.THRESH_BINARY
    )
    return img2, img


def img_contours(img, psize=.1):
    size = int(psize * np.mean(img.shape))
    contours, hierarchy = cv2.findContours(
        image=img, 
        mode=cv2.RETR_TREE, 
        method=cv2.CHAIN_APPROX_SIMPLE
    )
    bboxes = []
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        if w * h > size * size:
            bboxes.append([x, y, x + w, y + h])
    return bboxes


def draw_bboxes(img, boxes):
    for bbox in boxes:
        cv2.rectangle(
            img, 
            bbox[:2], 
            bbox[2:], 
            (255, 0, 0), 
            2
        )
    return img

In [ ]:
def img_bboxes(fnum, gap_l, gap_r, pkersize, psize, quantile):
    img1 = img_mean(
        files_names, 
        frame_num=fnum, 
        cover_l=gap_l, 
        cover_r=gap_r, 
        canny=False
    )
    img2 = get_img(
        file_path=f'{imgs_dir}/frame_{str_numbered(fnum)}.png', 
        canny=False
    )
    mask = get_img(
        file_path=f'{imgs_dir}/frame_{str_numbered(fnum)}.png', 
        canny=True
    )
    img_orig, img = img_diff(
        img1, img2, mask,
        psize=pkersize,
        quantile=quantile
    )
    bboxes = img_contours(img, psize=psize)
    return img_orig, img, bboxes

#### 3.2. Test for one frame

Result highly varies depending on the hyperparameters:

In [ ]:
GAP_L = 10
GAP_R = 10
PKERSIZE = .1
PSIZE = .1
QUANTILE = .95
FNUM = 20

img_orig, img, bboxes = img_bboxes(
    fnum=FNUM, 
    gap_l=GAP_L, 
    gap_r=GAP_R, 
    pkersize=PKERSIZE, 
    psize=PSIZE, 
    quantile=QUANTILE
)
img_orig = draw_bboxes(img_orig, bboxes)
fig = plt.figure(figsize=(16, 16))
fig.add_subplot(1, 2, 1)
plt.imshow(img)
fig.add_subplot(1, 2, 2)
plt.imshow(img_orig)
plt.show()

#### 3.3. Process many frames

In [ ]:
all_bboxes = []
for frame_num in tqdm(range(15, 35)):
    img_orig, img, bboxes = img_bboxes(
        fnum=frame_num, 
        gap_l=GAP_L, 
        gap_r=GAP_R, 
        pkersize=PKERSIZE, 
        psize=PSIZE, 
        quantile=QUANTILE
    )
    all_bboxes.append(bboxes)

In [ ]:
for bboxes in all_bboxes:
    img_orig = draw_bboxes(img_orig, bboxes)
plt.imshow(img_orig)
plt.show()

### 4. Write results to video

In [ ]:
out = cv2.VideoWriter(
    f'{WORK_PATH}/{OUT_FILE}',
    cv2.VideoWriter_fourcc(*'XVID'), 
    c5,
    (int(c3), int(c4)),
    True
)
trace = []
for i, frame_num in enumerate(tqdm(range(15, 35))):
    file_path = f'{imgs_dir}/frame_{str_numbered(frame_num)}.png'
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = draw_bboxes(img, all_bboxes[i])
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    out.write(img)
out.release()

### 5. Summary

#### + Pros +

1. Easy to implement, basic math functions only
2. Fast computations, can run e.g. on Rasberry PI or other lightweight devices

#### - Cons -

1. Highly depends on hyperparameters and should be tuned for every video
2. Can generate many false positives
3. Generally not works...

Should we try neural networks instead?